In [1]:
# Useful functions (covered in previous sections of the course)
import hashlib

def hash256(data: bytes):
    '''Two rounds of SHA256 (aka Hash256)'''
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(data: bytes):
    '''sha256 followed by ripemd160'''
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.new('ripemd160', hash_1).digest()
    return hash_2

# Bitcoin addresses

Here we will cover the different types of bitcoin addresses as well as how they can be encoded and decoded.

## Introduction
When Alice sends Bob bitcoin, Alice does so by creating a new transaction where one (or more) of the outputs has a scriptPubkey (aka 'locking script') specified by Bob. What makes the output effectively 'belong' to Bob is that only he knows how to create a scriptSig (aka 'unlocking script') which will satisfy or 'unlock' the locking script.

If Bob were to send Alice the scriptPubkey as raw data, any error in communication could result in Alice sending the bitcoin to the wrong scriptPubkey and having the bitcoins effectively impossible to recover.

To solve this problem, there are common standards for encoding scriptPubkeys as addresses. These addresses are designed to be easier to read, specify which network the address is intended for (mainnet/testnet), and contain a checksum to help with error detection. A full list of bitcoin address prefixes can be found here: https://en.bitcoin.it/wiki/List_of_address_prefixes

Note that many forks of bitcoin (such as litecoin) use similar address standards but with different prefixes to differentiate between them. If a wallet implementation doesn't check that the prefix matches with the type of transaction being created, the user may think they are sending litecoin to their address, when it is actually sending bitcoin. 

Note that the addresses themselves are never stored on the blockchain. However, bitcoind and other bitcoin explorers will often display a transaction's output scriptPubkeys as address for readability. 

Below we will cover the current address formats that exists with examples of how to encode/decode the most commonly used
scriptPubkeys.

- Base58
    - p2pkh
    - p2sh
    - p2sh-p2wpkh
- Bech32
    - p2wpkh
    - p2wsh
- Bech32m
    - p2tr

## Base58

In [2]:
import base58

def encode_base58_checksum(b: bytes):
    return base58.b58encode(b + hash256(b)[:4]).decode()

def decode_base58(s: str):
    return base58.b58decode(s)

# TODO
# def validate_checksum()

def pk_to_p2pkh(compressed: bytes, network: str):
    '''Creates a p2pkh address from a compressed pubkey'''
    pk_hash = hash160(compressed)
    if network == "regtest" or network == "testnet":
        prefix = bytes.fromhex("6f")
    elif network == "mainnet":
        prefix = bytes.fromhex("00")
    else:
        return "Enter the network: testnet/regtest/mainnet"
    return encode_base58_checksum(prefix + pk_hash)

def script_to_p2sh(redeemScript, network):
    rs_hash = hash160(redeemScript)
    if network == "regtest" or network == "testnet":
        prefix = bytes.fromhex("c4")
    elif network == "mainnet":
        prefix = bytes.fromhex("05")
    else:
        return "Enter the network: tesnet/regtest/mainnet"
    return encode_base58_checksum(prefix + rs_hash)

### Examples

#### Creating a base58 address from a pubkey
Given the pubkey `02466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27`, create a p2pkh address.

In [3]:
pubkey = bytes.fromhex("02466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27")
p2pkh_address = pk_to_p2pkh(pubkey, 'regtest')
print(p2pkh_address)

mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


#### Decoding a base58 address
Now from the base58 address, let's decode the prefix and scriptPubkey.

In [4]:
address = 'mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug'
address_decoded = decode_base58(address)

# Check the checksum is valid
decoded = address_decoded[:-4] # everything before the last 4 bytes is the message
checksum = address_decoded[-4:] # last 4 bytes are the checksum

# Check that the first four bytes of the hash are equal to the checksum
print(hash256(decoded)[:4] == checksum)

print("prefix: ", hex(decoded[0]))

pk_hash = decoded[1:]
print("pubkey hash: ", pk_hash.hex())

True
prefix:  0x6f
pubkey hash:  531260aa2a199e228c537dfa42c82bea2c7c1f4d


#### Pubkey hash to scriptPubkey
Now that we have the pubkey hash, to turn it into a scriptPubkey we need to put it into the standard p2pkh script:

`OP_DUP OP_HASH160 <pubKeyHash> OP_EQUALVERIFY OP_CHECKSIG`

We can look up the corresponding op codes from https://en.bitcoin.it/wiki/Script

Note that in front of `<pubKeyHash>` we need to add an opcode for the length of the hash. Since the pubkey hash is taken from hash160, we have a 20 byte hash, which is `14` in hex.

In [5]:
scriptPubkey = bytes.fromhex("1976a914" + pk_hash.hex() + "88ac")
print("scriptPubkey: ", scriptPubkey.hex())

scriptPubkey:  1976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac


## Segwit

In [6]:
# Segwit
def pk_to_p2wpkh(compressed, network):
    '''generates a p2wpkh bech32 address corresponding to a compressed pubkey'''
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    spk = binascii.unhexlify(redeemScript.hex())
    version = spk[0] - 0x50 if spk[0] else 0
    program = spk[2:]
    if network == "testnet":
        prefix = 'tb'
    if network == "regtest":
        prefix = 'bcrt'
    elif network == "mainnet":
        prefix = 'bc'
    else:
        return "Enter the network: testnet/regtest/mainnet"
    return bech32.encode(prefix, version, program)

def pk_to_p2sh_p2wpkh(compressed, network):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex("0014"+str(pk_hash.hex()))
    rs_hash = hash160(redeemScript)
    if network == "regtest" or network == "testnet":
        prefix = b"\xc4"
    elif network == "mainnet":
        prefix = b"\x05"
    else:
        return "Enter the network: tesnet/regtest/mainnet"
    return encode_base58_checksum(prefix + rs_hash)

# def script_to_p2wsh()

In [7]:
# Taproot
# def pk_to_p2tr()

## Quiz

What is the scriptPubkey encoded by this address `<address>`? What network is it intended for (mainnet/testnet)?

